In [1]:
import pandas as pd
df=pd.read_csv('real+generated_data.csv')

In [2]:
df

,company_name,price,departure_time,co2_emission,stops,days_left,Source_city,Destination_city,duration,year,month,day
0,IndiGo,3613.000000,345,62,0,1,HYDERABAD,MUMBAI,90,2024,9,5
1,Air India,3904.000000,800,68,0,1,HYDERABAD,MUMBAI,90,2024,9,5
2,Akasa Air,4020.000000,705,61,0,1,HYDERABAD,MUMBAI,105,2024,9,5
3,IndiGo,3746.000000,410,62,0,1,HYDERABAD,MUMBAI,85,2024,9,5
4,Air India,3904.000000,500,68,0,1,HYDERABAD,MUMBAI,95,2024,9,5
...,...,...,...,...,...,...,...,...,...,...,...,...
407413,IndiGo,3663.527487,731,65,0,9,HYDERABAD,MUMBAI,78,2024,9,10
407414,IndiGo,3683.257757,414,59,0,27,HYDERABAD,MUMBAI,84,2024,9,16
407415,SriLankan,21336.751322,377,368,1,14,MUMBAI,DELHI,469,2024,9,17
407416,"Emirates, flydubai",30893.899267,936,349,1,24,HYDERABAD,DELHI,1032,2024,9,16


In [3]:
df_encoded = pd.get_dummies(df, columns=['company_name', 'Source_city', 'Destination_city'])

In [4]:
df_encoded

,price,departure_time,co2_emission,stops,days_left,duration,year,month,day,company_name_Air India,...,company_name_Vistara,company_name_VistaraAir India,company_name_flydubai,"company_name_flydubai, Emirates",Source_city_DELHI,Source_city_HYDERABAD,Source_city_MUMBAI,Destination_city_DELHI,Destination_city_HYDERABAD,Destination_city_MUMBAI
0,3613.000000,345,62,0,1,90,2024,9,5,False,...,False,False,False,False,False,True,False,False,False,True
1,3904.000000,800,68,0,1,90,2024,9,5,True,...,False,False,False,False,False,True,False,False,False,True
2,4020.000000,705,61,0,1,105,2024,9,5,False,...,False,False,False,False,False,True,False,False,False,True
3,3746.000000,410,62,0,1,85,2024,9,5,False,...,False,False,False,False,False,True,False,False,False,True
4,3904.000000,500,68,0,1,95,2024,9,5,True,...,False,False,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407413,3663.527487,731,65,0,9,78,2024,9,10,False,...,False,False,False,False,False,True,False,False,False,True
407414,3683.257757,414,59,0,27,84,2024,9,16,False,...,False,False,False,False,False,True,False,False,False,True
407415,21336.751322,377,368,1,14,469,2024,9,17,False,...,False,False,False,False,False,False,True,True,False,False
407416,30893.899267,936,349,1,24,1032,2024,9,16,False,...,False,False,False,False,False,True,False,True,False,False


In [5]:
# Split the data into features and target variable
X = df_encoded.drop('price', axis=1)  # Features (all columns except the target)
y = df_encoded['price']  # Target variable

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2024-11-02 18:10:31.389545: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-02 18:10:31.483777: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-02 18:10:34.596906: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
# Custom R² metric function
def r2_metric(y_true, y_pred):
    ss_res = tf.reduce_sum(tf.square(y_true - y_pred))
    ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return 1 - ss_res / (ss_tot + tf.keras.backend.epsilon())

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),  # Dropout to prevent high accuracy
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dense(1)  # Output layer for regression
])

model.compile(optimizer='adam', loss='mse', metrics=[r2_metric])  # Use custom R² metric






/home/dinesh/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-02 18:10:44.366718: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-02 18:10:44.368813: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the requi

In [9]:
early_stopping = EarlyStopping(monitor='val_r2_metric', patience=5, restore_best_weights=True, mode='max')

# Train the model with early stopping
history = model.fit(
    X_train, y_train, epochs=25, batch_size=32,
    validation_data=(X_test, y_test), verbose=1,
    callbacks=[early_stopping]
)

Epoch 1/25
10186/10186 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step - loss: 79799312.0000 - r2_metric: 0.5187 - val_loss: 29064652.0000 - val_r2_metric: 0.8337
Epoch 2/25
10186/10186 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - loss: 36714876.0000 - r2_metric: 0.7816 - val_loss: 29855984.0000 - val_r2_metric: 0.8328
Epoch 3/25
10186/10186 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - loss: 35835220.0000 - r2_metric: 0.7857 - val_loss: 30148854.0000 - val_r2_metric: 0.8313
Epoch 4/25
10186/10186 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 35824988.0000 - r2_metric: 0.7869 - val_loss: 30979618.0000 - val_r2_metric: 0.8280
Epoch 5/25
10186/10186 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - loss: 34578180.0000 - r2_metric: 0.7971 - val_loss: 37359264.0000 - val_r2_metric: 0.7934
Epoch 6/25
10186/10186 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - loss: 33541210.0000 - r2_metric: 0.8009 - val_loss: 43883404.0000 - val_r2_metric: 0.7579


In [10]:
# Evaluate the model
loss, r2 = model.evaluate(X_test, y_test)
print(f"R² on test data: {r2:.4f}")

1297/1297 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 11013737.0000 - r2_metric: 0.9115
R² on test data: 0.9085
